In [ ]:
"""
    creates a windowed dataset from 10-second EEG epochs. special case 2 not allowed. otherwise, labels are assigned to the center epoch in the window.
"""
## todo wn not most convenient...saves n-1 copies of each epoch..
from tqdm import tqdm
from lib.utils import load_raw_list
import os
import torch
from sklearn.model_selection import train_test_split
from torch.nn.functional import one_hot
import json
import random
normalize = True
balance = False
windowsize = 1
train_size = .5
dev_size = .25
test_size = .25
files = list(range(32))
n_holdout_files = 5
holdout_fileindices = random.sample(files,n_holdout_files)

y_train_all = torch.Tensor()
y_dev_all = torch.Tensor()
y_test_all = torch.Tensor()
x_train_i = 0
x_dev_i = 0
x_test_i = 0
data_dir = f'w{windowsize}'
os.makedirs(data_dir)
os.makedirs(f'{data_dir}/train')
os.makedirs(f'{data_dir}/dev')
os.makedirs(f'{data_dir}/test')
config = {
    'TRAIN_SIZE':train_size,
    'DEV_SIZE':dev_size,
    'TEST_SIZE':test_size,
    'FILES':files,
    'NORMALIZED':normalize,
    'BALANCED':balance,
    'HOLDOUT_FILES':holdout_fileindices
}
with open(f'{data_dir}/config.json', 'w') as f:
     f.write(json.dumps(config))

for fileindex in tqdm(files):
    X,y = load_raw_list([fileindex])

    if(normalize):
        # center, stretch
        X = (X - X.mean(axis=1,keepdim=True))/X.std(axis=1,keepdim=True)
        # drop row if any element is inf
        not_inf_idx = torch.where(~X.isinf().any(axis=1))[0]
        X,y = X[not_inf_idx], y[not_inf_idx]
    if(windowsize == 1):
        windowsize = 1
    else:
        # windowing
        # only works for odd windows, puts label at center
        cat = [X[:-(windowsize-1)]]
        for i in range(1,(windowsize-1)):
            cat.append(X[i:i-(windowsize-1)])
        cat.append(X[(windowsize-1):])
        X = torch.cat(cat,axis=1).float()
        y = y[windowsize//2:-(windowsize//2)]
    if (fileindex in holdout_fileindices):
        torch.save((X,y),f'{data_dir}/holdout_{fileindex}.pt')
        continue
    # train test split
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=(1-train_size),shuffle=True,stratify=y,random_state=0)
    X_dev,X_test,y_dev,y_test = train_test_split(X_test,y_test,test_size=(test_size/(1-train_size)),shuffle=True,stratify=y_test,random_state=0)

    y_train_all = torch.cat([y_train_all,y_train])
    y_dev_all = torch.cat([y_dev_all,y_dev])
    y_test_all = torch.cat([y_test_all,y_test])

    for Xi in X_train:
        torch.save(Xi.clone(),f'{data_dir}/train/{x_train_i}.pt')
        x_train_i += 1
    for Xi in X_dev:
        torch.save(Xi.clone(),f'{data_dir}/dev/{x_dev_i}.pt')
        x_dev_i += 1
    for Xi in X_test:
        torch.save(Xi.clone(),f'{data_dir}/test/{x_test_i}.pt')
        x_test_i += 1
torch.save(y_train_all,f'{data_dir}/y_train.pt')
torch.save(y_dev_all,f'{data_dir}/y_dev.pt')
torch.save(y_test_all,f'{data_dir}/y_test.pt')

In [ ]:
# test if worked
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader
trainloader = DataLoader(Dataset2p0(dir=f'{data_dir}/train/',labels=f'{data_dir}/y_train.pt'),batch_size=1,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{data_dir}/dev/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
testloader = DataLoader(Dataset2p0(dir=f'{data_dir}/test/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
X,y = next(iter(trainloader))
X,y = next(iter(devloader))
X,y = next(iter(testloader))
print(X.shape,y.shape)
print(len(trainloader)+len(devloader)+len(testloader))